## This notebook will walk through a common pipeline for calculating a developmental index from an expression matrix containing normalized expression data (i.e. RPKM, FPKM, or TPM). The input dataframe should be formatted such that every column corresponds to an individual sample/cell, and every row corresponds to a gene.
---

Step (1) - Import the necessary packages and data. Clean data as necessary.

---

In [15]:
import pandas as pd
import numpy as np

import developmental_index as dvp

df = pd.read_csv('C:\\Users\\Ben\\Dropbox\\bilbo_lab_spr2020\\microglia-seq_website\\microglia-seq\\mdi_w_rpkm\\GSE99622_hanamsagar2017_tpm_unmelted_v2.csv', header = [0, 1, 2, 3, 4])
genes = df.iloc[:, df.columns.get_level_values(4) == 'gene'].values.flatten()
df.set_index(genes, inplace = True)
df.drop(df.columns[0], axis = 1, inplace = True)

---

Step (2) - Scale the expression values to between 0 and 1, so that all genes contribute to the index equally

---

In [16]:
df = dvp.scale_expression(df)

--- 

Step (3) - Drop any rows (genes) that do not have detectable expression in any of the samples

---

In [17]:
df = dvp.drop_unexpressed_genes(df)

---

Step (4) - Extract the columns (samples) corresponds to all the 'young' and 'old' samples, so that they can be compared against one another to determine if there is a significant difference in expression from young to old

---

In [18]:
## these lines are different based on how you have indexed the samples in your dataframe. You may need to adjust accordingly
young = df['E18']
old = df['P60']
old = old.iloc[:, old.columns.get_level_values(1)=='SAL']

---

Step (4.5) - Use the 'identify significant genes' function to identify the genes that are regulated by development. Here you need to specify young and old columns, which we defined in the cell - Step 4 - above

---

In [19]:
df = dvp.identify_significant_genes(df, young, old)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


---

Step (5) - Remove any rows (genes) that were not regulated by development, i.e. insignificant

---

In [20]:
df = dvp.remove_insignificant_rows(df)

AttributeError: module 'developmental_index' has no attribute 'remove_insignificant_rows'

---

OPTIONAL STEP - Create a sub-dataframe containing just useful regulated gene information (direction and valence)


---

In [ ]:
regulated_genes = dvp.extract_regulated_genes(df)

In [ ]:
regulated_genes

---

Step (6) - Use the index to generate index values for each individual sample (column) in your dataset. Here you will need to also provide the index location for sample columns (usually df.columns[1:-4]).

---

In [ ]:
index = dvp.generate_index(df, sample_cols = df.columns[1:-4])

---

Step (7) - Scale all index values to between 0 and 1, with 1 being most mature, and 0 least mature

---

In [ ]:
index = dvp.scale_index(index)

---

OPTIONAL STEP - Cleaning up output columns, checking that everything worked properly, and exporting as .csv

---

In [ ]:
index.drop('level_4', axis = 1, inplace = True)
index

In [ ]:
index.to_csv('calculated_index.csv')
regulated_genes.to_csv('regulated_genes.csv')